# 모듈

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from platform import python_version

# random 고정시 필요한 모듈
import os
import random

# 모델 형성시 필요한 모듈
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import metrics

# local적인 모델 해석 
import lime
import lime.lime_tabular

# random 고정
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
random.seed(42)

In [2]:
print(f'python version: {python_version()}')
print(f'numpy version : {np.__version__}')
print(f'pandas version : {pd.__version__}')
print(f'matplotlib version : {matplotlib.__version__}')
print(f'sklearn version : {sklearn.__version__}')
print(f'tensorflow version : {tf.__version__}')

python version: 3.7.4
numpy version : 1.18.5
pandas version : 1.1.1
matplotlib version : 3.3.1
sklearn version : 0.23.2
tensorflow version : 2.3.0


# train 데이터 가공

In [3]:
# 파일을 저장한 위치를 써 주세요.
directory = os.getcwd()

In [4]:
# training 에 필요한 데이터를 불러옵니다.
X_df=pd.read_excel(directory+'/X_for_train.xlsx') 

In [5]:
X = X_df.copy() ; X

,Unnamed: 0,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,...,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드,타 채널 시청자 수 평균,가전제품,농수축소분류,어류손질여부
0,0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000,10.0,...,프라임아님,1,0,0,0,0,1520.0,가전제품 아님,분류에없음,해당없음
1,1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000,10.0,...,프라임아님,0,1,0,0,0,1520.0,가전제품 아님,분류에없음,해당없음
2,2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000,10.0,...,프라임아님,1,0,0,0,0,1520.0,가전제품 아님,분류에없음,해당없음
3,3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000,10.0,...,프라임아님,0,1,0,0,0,1520.0,가전제품 아님,분류에없음,해당없음
4,4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000,10.0,...,프라임아님,1,0,0,0,0,1520.0,가전제품 아님,분류에없음,해당없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000,5.0,...,프라임아님,0,0,0,1,1,1335.9,가전제품 아님,분류에없음,해당없음
35375,35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000,5.0,...,프라임아님,0,0,1,0,1,1520.0,가전제품 아님,분류에없음,해당없음
35376,35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000,5.0,...,프라임아님,0,0,0,1,1,1520.0,가전제품 아님,분류에없음,해당없음
35377,35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000,5.0,...,프라임아님,0,0,1,0,1,1520.0,가전제품 아님,분류에없음,해당없음


In [6]:
# target data 를 빼냅니다.
y=X['취급액']

In [7]:
# 예측에 쓰지 않는 열 제거
X.drop(columns = ['Unnamed: 0','취급액','방송일시','마더코드','상품코드','상품명','년','일','시분','월일','시간열','정수노출(분)','요일/시간','날짜','시각'],inplace = True)

In [8]:
# 더미화
X = pd.get_dummies(X,columns=['월','요일','분기','상품군','PrimeTime','어류손질여부','가전제품','농수축소분류'])

In [9]:
# X_col 의 이름 저장
X_features=X.columns

In [10]:
# scaling 과정
# 큰 skewness 를 가지는 값들은 대부분 애초에 분포가 너무 틀어져있어 (노출 분은 20/10분이 거의다.) log/squre 등의 변환을 해도 똑같을거라 판단
# scaling 을 하는것으로 타협을 보았다.
# 그리고, 더미값을 가지는 경우에도 scaling 을 해 주어서 평균 0 / 분산 1 을 맞추어주는게 나중에 DNN 이 학습을 더 잘할 거라 판단하였다.(SELU 의 경우 모든 특성이 0,1 이여야한다.)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
X

array([[-0.10967928, -0.55549003, -0.20858117, ..., -0.06437268,
        -0.05297285, -0.08298992],
       [-0.10967928, -0.55549003, -0.20858117, ..., -0.06437268,
        -0.05297285, -0.08298992],
       [-0.10967928, -0.55549003, -0.20858117, ..., -0.06437268,
        -0.05297285, -0.08298992],
       ...,
       [-0.10967928, -0.35098812, -0.92228108, ..., -0.06437268,
        -0.05297285, -0.08298992],
       [-0.10967928, -0.36695236, -0.92228108, ..., -0.06437268,
        -0.05297285, -0.08298992],
       [-0.10967928, -0.3829166 , -0.92228108, ..., -0.06437268,
        -0.05297285, -0.08298992]])

In [11]:
# dataset train/test set 으로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [12]:
# MAPE 정의
def MAPE(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# DNN 모델 training

In [ ]:
# random 고정
tf.random.set_seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
random.seed(42)

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1, 
    inter_op_parallelism_threads=1)

sess = tf.compat.v1.Session(
    graph=tf.compat.v1.get_default_graph(), 
    config=session_conf)

tf.compat.v1.keras.backend.set_session(sess)

# 모델 훈련
model1 = keras.models.Sequential()
model1 = Sequential([
    Dense(128, kernel_initializer='normal', activation = "relu", input_shape=X_train.shape[1:]), 
    Dropout(0.2),
    Dense(256, kernel_initializer='normal', activation = "relu"),
    Dropout(0.2),
    Dense(512, kernel_initializer='normal', activation = "relu"),
    Dropout(0.2),
    Dense(1024, kernel_initializer='normal', activation = "relu"),
    Dropout(0.2),
    Dense(512, kernel_initializer='normal', activation = "relu"),
    Dropout(0.2),
    Dense(256, kernel_initializer='normal', activation = "relu"),
    Dropout(0.2),
    Dense(32, kernel_initializer='normal', activation = "relu"),
    Dense(1, kernel_initializer='normal'), ])

model1.compile(loss="mape",  # 평가기준이 mape 이니까 이걸로 하자.
              optimizer=keras.optimizers.Adam())
checkpoint = keras.callbacks.ModelCheckpoint(filepath="predict_model1.h5", #저장할 모델 이름
                                             monitor = 'val_loss', #monitoring 할 기준
                                             save_best_only=True ) #
early_stopping = keras.callbacks.EarlyStopping(patience=16, 
                                             restore_best_weights=True)
history1 = model1.fit(X_train, y_train, 
                        epochs=160,
                        validation_data=(X_test, y_test),
                        callbacks=[checkpoint,early_stopping])
model1 = keras.models.load_model(filepath = "predict_model1.h5")
evaluation1 = model1.evaluate(X_test, y_test)
# 39.7790 의 값이 나왔다.

Epoch 1/160
885/885 [==============================] - 9s 11ms/step - loss: 64.0785 - val_loss: 48.4956
Epoch 2/160
885/885 [==============================] - 9s 10ms/step - loss: 47.7901 - val_loss: 47.3003
Epoch 3/160
885/885 [==============================] - 10s 11ms/step - loss: 46.2321 - val_loss: 45.0442
Epoch 4/160
885/885 [==============================] - 11s 12ms/step - loss: 44.7950 - val_loss: 44.0305
Epoch 5/160
885/885 [==============================] - 10s 11ms/step - loss: 43.7266 - val_loss: 44.4475
Epoch 6/160
  1/885 [..............................] - ETA: 0s - loss: 35.8103

In [ ]:
# random 고정
tf.random.set_seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
random.seed(42)

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1, 
    inter_op_parallelism_threads=1)

sess = tf.compat.v1.Session(
    graph=tf.compat.v1.get_default_graph(), 
    config=session_conf)

tf.compat.v1.keras.backend.set_session(sess)

# 모델 훈련
model2 = keras.models.Sequential()
model2.add(keras.layers.Dense(200, activation="selu",kernel_initializer="lecun_normal",input_shape=X_train.shape[1:]))
model2.add(Dropout(0.2))
for layer in range(5):
    model2.add(keras.layers.Dense(200, activation="selu",kernel_initializer="lecun_normal"))
    model2.add(Dropout(0.2))
model2.add(keras.layers.Dense(100, activation="selu",kernel_initializer="lecun_normal"))
model2.add(Dropout(0.2))
model2.add(keras.layers.Dense(10))
model2.add(keras.layers.Dense(1))
model2.compile(loss="mape",  # 평가기준이 mape 이니까 이걸로 하자.
              optimizer=keras.optimizers.Adam())
checkpoint = keras.callbacks.ModelCheckpoint(filepath="predict_model2.h5", # 모델을 저장합니다.
                                             monitor = 'val_loss', #monitoring 할 기준
                                             save_best_only=True ) # 
early_stopping = keras.callbacks.EarlyStopping(patience=24, #2 만 줘보자.
                                             restore_best_weights=True)

history2 = model2.fit(X_train, y_train, 
                        epochs=160,
                        validation_data=(X_test, y_test),
                        callbacks=[checkpoint,early_stopping])
model2 = keras.models.load_model(filepath = "predict_model2.h5") # 위에서 학습한 모델을 불러옵니다.
evaluation2 = model2.evaluate(X_test, y_test) 
# 39.4675 의 값이 나왔다.

In [ ]:
# random 고정
tf.random.set_seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
random.seed(42)

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1, 
    inter_op_parallelism_threads=1)

sess = tf.compat.v1.Session(
    graph=tf.compat.v1.get_default_graph(), 
    config=session_conf)

tf.compat.v1.keras.backend.set_session(sess)

model3 = keras.models.Sequential()
model3 = Sequential([
    Dense(128, kernel_initializer='he_normal', activation = "elu", input_shape=X_train.shape[1:]),
    Dropout(0.2),
    Dense(256, kernel_initializer='he_normal', activation = "elu"),
    Dropout(0.2),
    Dense(512, kernel_initializer='he_normal', activation = "elu"),
    Dropout(0.2),
    Dense(512, kernel_initializer='he_normal', activation = "elu"),
    Dropout(0.2),
    Dense(512, kernel_initializer='he_normal', activation = "elu"),
    Dropout(0.2),
    Dense(512, kernel_initializer='he_normal', activation = "elu"),
    Dropout(0.2),
    Dense(512, kernel_initializer='he_normal', activation = "elu"),
    Dropout(0.2),
    Dense(256, kernel_initializer='he_normal', activation = "elu"),
    Dropout(0.2),
    Dense(32, kernel_initializer='he_normal', activation = "elu"),
    Dropout(0.2),
    Dense(1, kernel_initializer='he_normal'), 
])

model3.compile(loss="mape",  # 평가기준이 mape 이니까 이걸로 하자.
              optimizer=keras.optimizers.Adam())
checkpoint = keras.callbacks.ModelCheckpoint(filepath="predict_model3.h5", #저장할 모델 이름
                                             monitor = 'val_loss', #monitoring 할 기준
                                             save_best_only=True ) # 
early_stopping = keras.callbacks.EarlyStopping(patience=16, #2 만 줘보자.
                                             restore_best_weights=True)

history3 = model3.fit(X_train, y_train, 
                        epochs=160,
                        validation_data=(X_test, y_test),
                        callbacks=[checkpoint,early_stopping])
model3 = keras.models.load_model(filepath = "predict_model3.h5")
evaluation3 = model3.evaluate(X_test, y_test) 

# 39.3965의 값이 나왔다.

In [ ]:
# 다른 모델들을 합쳐 하나의 모델로 앙상블한다.
# test 를 통해 어느정도의 성능이 나오는지 체크
y_pred = 0.3 * model1.predict(X_test) + 0.35 * model2.predict(X_test) + 0.35 * model3.predict(X_test)
MAPE(y_test,y_pred.reshape(-1))
# 38.68 의 값이 나왔다.

# 예측 데이터 가공

In [ ]:
# 처리된 X 예측 데이터. 더미화까지 진행되어있는 상태이다.
X_pred = pd.read_excel('X_for_predict.xlsx')

In [ ]:
# 예측에 쓰지 않는 열 제거
X_pred.drop(columns = ['Unnamed: 0','방송일시','마더코드','상품코드','상품명','년','일','시분','월일','시간열','정수노출(분)','요일/시간','날짜','시각'],inplace = True)

In [ ]:
# 열 이름 저장
X_pred_features = X_pred.columns

In [ ]:
# scaling 은 앞에서 train 의 scailing 을 따른다.
X_pred_scaled = scaler.transform(X_pred)

In [ ]:
# 예측값 형성
y_pred_ = 0.3 * model1.predict(X_pred_scaled) + 0.35 * model2.predict(X_pred_scaled) + 0.35*model3.predict(X_pred_scaled)

In [ ]:
# 코로나에 의한 가중치
constant = 66832554962.686/66337499000

In [ ]:
y_pred_ = constant * y_pred_

# 데이터 해석을 위한 xlsx 내보내기

In [ ]:
# predict 를 했던 x 데이터에 예측값을 붙인 뒤, 각 요일/ 각 카테고리 등 에 대한 상위/하위 데이터들을 조사해 어떤 특징이 있는지 살펴보았다.

In [ ]:
X_predict_data = pd.read_excel('X_for_predict.xlsx')

In [ ]:
X_predict_data['판애액예측'] = pd.DataFrame(y_pred_)

In [ ]:
X_predict_data.to_excel('데이터 해석.xlsx')

# 예측값 ecxel 로 형성하기

In [ ]:
# 이제 평가데이터 엑셀 파일에 예측값을 붙여주어야 한다.
y_pred_excel=pd.read_excel('2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx',header=1)

In [ ]:
y_pred_excel.drop(columns = ['취급액'],inplace=True)

In [ ]:
# 앞서 제품군이 무형/ 토요일 1800 ~ 1820 에 시작하는 데이터는 예측에 쓰지 않기 때문에, index 를 새로 불러온다.
y_pred_index = pd.read_csv('predict_index.csv')

In [ ]:
# 새 index 레 우리의 예측값을 붙여준다.
y_pred_index['취급액'] = pd.DataFrame(y_pred_)
y_pred_index.index = y_pred_index['index']
y_pred_index.drop(columns=['Unnamed: 0','index'],inplace=True)

In [ ]:
# 그리고, 그 index 에 맞추어 merge 해 주어서 엑셀을 형성한다.
pred_final=pd.merge(y_pred_excel, y_pred_index, how='outer', left_index=True, right_index=True) 
pred_final

In [ ]:
pred_final.to_excel('6월 판매실적 예측.xlsx')

# Lime 을 통한 해석

In [ ]:
def predict(x):
    pred = 0.3*model1.predict(x)+ 0.35*model2.predict(x)+ 0.35*model3.predict(x)
    return pred.flatten()

In [ ]:
X_features

In [ ]:
# 위 한글이름의 값을 영어로 바꾸어주어야 lime 에서 출력이 제대로 되기 떄문에 바꾸어주었다.
X_features_ENG=['expsr(min)', 'unit_price', 'avg_cast_min', 'overlap_num', 'avg_ratings', '06:00~08:00', '08:00~10:00',
       '10:00~12:00', '12:00~14:00', '14:00~16:00', '16:00~18:00',
       '18:00~20:00', '20:00~22:00', '22:00~00:00', '00:00~02:00',
       '02:00~04:00', 'rainfall(mm)', 'avg_temp(℃)', 'min_temp(℃)', 'max_temp(℃)', 'celebrity', 'income',
       'holiday', 'type_power', 'CPI', 'RS', 'male', 'female', 'no_int', 'lump_sum_pay', 'brand',
       'avgviewer_othr_chnnls', 'Jan', 'Feb', 'March', 'April', 'May', 'June', 'July', 'Aug',
       'Sep', 'Oct', 'Nov', 'Dec', 'Fri', 'Thurs', 'Wed', 'Mon', 'Sun',
       'Sat', 'Tue', '1Q', '2Q', '3Q', '4Q', 'type_furn',
       'type_home_elec', 'type_health', 'type_agri_fish_lvstck', 'type_daily_gds', 'type_underclth', 'type_clth',
       'type_beauty', 'type_miscll', 'type_kitch', 'type_bed', 'PrimeTime_other',
       'PrimeTime_food', 'PrimeTime_morning', 'not_PrimeTime', 'fishtrim_0',
       'fishtrim_1', 'fishtrim_NA', 'home_elec_TV', 'not_home_elec ', 'home_elec_drier',
       'home_elec_airclean', 'home_elec_fridge', 'home_elec_notebk', 'home_elec_wrlss_vcmclnr', 'home_elec_washmch',
       'home_elec_aircon', 'home_elec_clothmch', 'agro_driedfish', 'agro_fruit', 'agro_kimchi',
       'agro_not_cat', 'agro_rice', 'agro_fish', 'agro_meat', 'agro_drink',
       'agro_sauce/spice', 'agro_snacks']

In [ ]:
# set up the LIME explainer
np.random.seed(42)
explainer = lime.lime_tabular.LimeTabularExplainer(X_train,
                                                  training_labels = y_train,
                                                  feature_names = X_features_ENG,
                                                  mode = 'regression',
                                                  discretize_continuous = False)

In [ ]:
# lime 은 아래와 같이 , local 한 데이터가 긍정/ 부정 영향인지, 얼마나 예측에 영향을 끼치는지 말해준다.
np.random.seed(42)
exp = explainer.explain_instance(X_pred_scaled[0],
                                 predict,
                                 num_features=20,
                                 distance_metric='euclidean',
                                 num_samples=1000)

plt.figure(figsize=(40,5))
exp.show_in_notebook(show_table=True, predict_proba=True, show_predicted_value=True)

In [ ]:
# 각 20개의 상위 영향을 가진 값을 표출한다.
def expl(x):
    np.random.seed(42)
    exp = explainer.explain_instance(X_pred_scaled[x],
                                 predict,
                                 num_features=20,
                                 distance_metric='euclidean',
                                 num_samples=1000)
    print(exp.predicted_value)
    exp.as_pyplot_figure() ;

In [ ]:
# 6시~8시 상위 
expl(1246)
expl(73)
expl(1245)

In [ ]:
# 8:00 ~ 10:00 상위
expl(1787)
expl(1249)
expl(2396)

In [ ]:
# 10:00 ~ 12:00 상위
expl(579)
expl(2399)
expl(973)

In [ ]:
# 12:00 ~ 14:00 상위
expl(1177)
expl(2581)
expl(594)

In [ ]:
# 14:00 ~ 16:00 상위
expl(1195)
expl(1305)
expl(1285)

In [ ]:
# 16:00 ~ 18:00 상위
expl(1306)
expl(1933)
expl(1201)

In [ ]:
# 18:00 ~ 20:00 상위
expl(1321)
expl(2601)
expl(103)

In [ ]:
# 20:00 ~ 22:00 상위
expl(493)
expl(2543)
expl(1502)

In [ ]:
# 22:00 ~ 00:00 상위
expl(1863)
expl(2288)
expl(929)

In [ ]:
# 00:00 ~ 02:00 상위
expl(2478)
expl(1060)
expl(160)

In [ ]:
# 02:00 ~ 04:00 상위
expl(1604)
expl(547)
expl(2144)

In [ ]:
# 의류 상위
expl(1891)
expl(1156)
expl(1784)

In [ ]:
# 이미용 상위
expl(2189)
expl(1754)
expl(741)

In [ ]:
# 잡화 상위
expl(1813)
expl(248)
expl(1448)

In [ ]:
# 가구 상위
expl(644)
expl(645)
expl(646)

In [ ]:
# 가전 상위
expl(493)
expl(579)
expl(1321)

In [ ]:
# 건강기능 상위
expl(2116)
expl(2138)
expl(2114)

In [ ]:
# 농수축 상위
expl(1306)
expl(2543)
expl(1933)

In [ ]:
# 생활용품 상위
expl(1285)
expl(1364)
expl(634)

In [ ]:
# 속옷 상위
expl(2192)
expl(1664)
expl(1165)

In [ ]:
# 주방 상위
expl(1195)
expl(1787)
expl(2399)

In [ ]:
# 침구 상위
expl(809)
expl(808)
expl(807)

In [ ]:
# 월요일 상위
expl(2601)
expl(2598)
expl(2604)

In [ ]:
# 화요일 상위
expl(1502)
expl(2685)
expl(103)

In [ ]:
# 수요일 상위
expl(223)
expl(929)
expl(2189)

In [ ]:
# 목요일 상위
expl(2288)
expl(973)
expl(2261)

In [ ]:
# 금요일 상위
expl(2366)
expl(1754)
expl(1751)

In [ ]:
# 토요일 상위
expl(493)
expl(1195)
expl(1787)

In [ ]:
# 일요일 상위
expl(1306)
expl(2543)
expl(579)